In [4]:
from scipy.stats import multivariate_hypergeom

Set your 2 card combos and deck size here:

In [5]:
deck_size=41
x_1 = 6
x_2 = 2

Evaluate Multivariate Hypergeometric probabity density function:

The third field describes the "misses" in your hand.

In [6]:
_DRAW = 5
rv = multivariate_hypergeom(m=[x_1, x_2, deck_size-2], n=5)
rv.pmf(x=[1,1,3])

0.071494368420126